<a href="https://colab.research.google.com/github/Aparnamol-KS/CodeCompanion-GroqAI/blob/main/TinyLlama_QLoRA_Support_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Fine-Tune TinyLlama-1.1B with QLoRA on Health Data

In [ ]:
!pip install -q bitsandbytes accelerate datasets loralib peft transformers trl


In [ ]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
model.config.use_cache = False


In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import json

filename = 'health_dataset_500.json'

with open(filename, 'r') as file:
    data = json.load(file)


In [ ]:

dataset = Dataset.from_list(data)

def format_instruction(example):
    return f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"

dataset = dataset.map(lambda x: {"text": format_instruction(x)})


In [ ]:
def tokenize_function(example):
    tokenized = tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)


In [ ]:
training_args = TrainingArguments(
    output_dir="./tinyllama-qlora-support-bot",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_dir="./logs",
    num_train_epochs=3,
    logging_steps=10,
    save_total_limit=2,
    save_strategy="epoch",
    bf16=True,
    optim="paged_adamw_8bit"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

In [ ]:
model.save_pretrained("tinyllama-qlora-support-bot")
tokenizer.save_pretrained("tinyllama-qlora-support-bot")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r /content/tinyllama-qlora-support-bot /content/drive/MyDrive/tinybotphonesaved

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/drive/MyDrive/tinybotphonesaved"

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

instruction = "How to manage stress in my life?"
prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"

output = pipe(prompt, max_new_tokens=100)
print(output[0]['generated_text'])


In [ ]:
pip install -q gradio

In [ ]:
import gradio as gr
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

def format_prompt(instruction):
    return f"### Instruction:\n{instruction}\n\n### Response:\n"

def chat_response(message, chat_history):
    prompt = format_prompt(message)
    output = pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.7)
    generated_text = output[0]['generated_text']
    response = generated_text.split("### Response:\n")[-1].strip()
    chat_history.append((message, response))
    return "", chat_history

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🩺 Health Support Chatbot\nAsk anything related to wellness, healthy habits, sleep, stress, or self-care.")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your Question", placeholder="e.g., How to sleep better?", lines=1)
    clear = gr.Button("Clear Chat")

    msg.submit(chat_response, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

# Launch the app
demo.launch()